# Aggregate transcripts TPM and counts by gene

In [1]:
import os, time
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
from functools import reduce

### Import and merge all quant files from salmon's output

In [2]:
%%time
quants = []
namelist = None
tx_number = None

#import all quant.sf files
samples = sorted([s for s in os.listdir('quant/salmon_output') if s.startswith('SRR')])
for sample in samples:
    file = os.path.join('quant/salmon_output',sample,'quant.sf')
    quant_df = pd.read_csv(file, sep=r'\t', engine='python')
    #check all files have the same list of transcripts
    if namelist is None:
        namelist = quant_df.Name.values
    if not all(quant_df.Name.values == namelist):
        print(sample, 'has different transcript list')
    if tx_number is None:
        tx_number = len(quant_df)
    if len(quant_df) != tx_number:
        print(sample, 'has different transcript number')

    #drop length column as it won't be used
    quant_df.drop(['Length', 'EffectiveLength'], axis= 1, inplace= True)
    quant_df.rename(columns= {'NumReads':'counts'}, inplace= True)
    #add sample name to columns to facilitate merging
    quant_df.columns = [f'{col}_{sample}' if col != 'Name' else col for col in quant_df.columns]
    quants += [quant_df]

CPU times: user 16 s, sys: 467 ms, total: 16.4 s
Wall time: 16.4 s


In [3]:
%%time
#merge all quant files
merged = reduce(lambda left,right: pd.merge(left,right, on='Name', how='outer'), quants)
#sort columns alphabetically
merged = merged.reindex(['Name'] + sorted(merged.columns[1:], key=lambda x: x.lower()), axis=1)
print(merged.shape)
merged.head()

(207749, 29)
CPU times: user 2.87 s, sys: 285 ms, total: 3.15 s
Wall time: 3.16 s


In [4]:
#split all IDs in Name
IDs = merged.Name.str.split(pat='|', expand=True).drop([6,8], axis=1)
IDs.columns = ['Tx', 'Gene', 'ID2', 'ID3', 'ID4', 'ID5', 'Type']
IDs.head()

,Tx,Gene,ID2,ID3,ID4,ID5,Type
0,ENST00000456328.2,ENSG00000223972.5,OTTHUMG00000000961.2,OTTHUMT00000362751.1,DDX11L1-202,DDX11L1,processed_transcript
1,ENST00000450305.2,ENSG00000223972.5,OTTHUMG00000000961.2,OTTHUMT00000002844.2,DDX11L1-201,DDX11L1,transcribed_unprocessed_pseudogene
2,ENST00000488147.1,ENSG00000227232.5,OTTHUMG00000000958.1,OTTHUMT00000002839.1,WASH7P-201,WASH7P,unprocessed_pseudogene
3,ENST00000619216.1,ENSG00000278267.1,-,-,MIR6859-1-201,MIR6859-1,miRNA
4,ENST00000473358.1,ENSG00000243485.5,OTTHUMG00000000959.2,OTTHUMT00000002840.1,MIR1302-2HG-202,MIR1302-2HG,lincRNA


In [5]:
tx_quant = IDs.merge(merged, left_index=True, right_index=True)
tx_quant.head()

,Tx,Gene,ID2,ID3,ID4,ID5,Type,Name,counts_SRR6231076,counts_SRR6231077,counts_SRR6231078,counts_SRR6231079,counts_SRR6231080,counts_SRR6231081,counts_SRR6231082,counts_SRR6231083,counts_SRR6231084,counts_SRR6231085,counts_SRR6231086,counts_SRR6231087,counts_SRR6231088,counts_SRR6231089,TPM_SRR6231076,TPM_SRR6231077,TPM_SRR6231078,TPM_SRR6231079,TPM_SRR6231080,TPM_SRR6231081,TPM_SRR6231082,TPM_SRR6231083,TPM_SRR6231084,TPM_SRR6231085,TPM_SRR6231086,TPM_SRR6231087,TPM_SRR6231088,TPM_SRR6231089
0,ENST00000456328.2,ENSG00000223972.5,OTTHUMG00000000961.2,OTTHUMT00000362751.1,DDX11L1-202,DDX11L1,processed_transcript,ENST00000456328.2|ENSG00000223972.5|OTTHUMG000...,0.0,0.000,0.0,2.109,0.000,0.000,0.0,35.467,0.000,0.0,0.000,8.808,0.00,0.0,0.0,0.000000,0.0,0.094932,0.000000,0.000000,0.0,1.737810,0.000000,0.000000,0.000000,0.313647,0.000000,0.0
1,ENST00000450305.2,ENSG00000223972.5,OTTHUMG00000000961.2,OTTHUMT00000002844.2,DDX11L1-201,DDX11L1,transcribed_unprocessed_pseudogene,ENST00000450305.2|ENSG00000223972.5|OTTHUMG000...,0.0,0.000,0.0,0.000,3.243,1.874,0.0,4.744,0.000,0.0,1.128,0.000,0.00,0.0,0.0,0.000000,0.0,0.000000,0.462135,0.370236,0.0,0.854551,0.000000,0.000000,0.166130,0.000000,0.000000,0.0
2,ENST00000488147.1,ENSG00000227232.5,OTTHUMG00000000958.1,OTTHUMT00000002839.1,WASH7P-201,WASH7P,unprocessed_pseudogene,ENST00000488147.1|ENSG00000227232.5|OTTHUMG000...,0.0,22.957,0.0,8.122,0.000,0.000,0.0,14.828,15.524,0.0,6.050,0.000,7.02,0.0,0.0,1.072521,0.0,0.467096,0.000000,0.000000,0.0,0.928311,1.270445,0.000000,0.309571,0.000000,0.308776,0.0
3,ENST00000619216.1,ENSG00000278267.1,-,-,MIR6859-1-201,MIR6859-1,miRNA,ENST00000619216.1|ENSG00000278267.1|-|-|MIR685...,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.000,0.000,1.0,0.000,0.000,0.00,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,16.239739,0.000000,0.000000,0.000000,0.0
4,ENST00000473358.1,ENSG00000243485.5,OTTHUMG00000000959.2,OTTHUMT00000002840.1,MIR1302-2HG-202,MIR1302-2HG,lincRNA,ENST00000473358.1|ENSG00000243485.5|OTTHUMG000...,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.00,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


### Aggregate TPM and counts by gene

In [9]:
TPM_cols = [col for col in tx_quant.columns if col.startswith('TPM')]
counts_cols = [col for col in tx_quant.columns if col.startswith('counts')]
agg = tx_quant.loc[:,['Tx','Gene']+TPM_cols+counts_cols].groupby('Gene').sum()
agg.head()

,TPM_SRR6231076,TPM_SRR6231077,TPM_SRR6231078,TPM_SRR6231079,TPM_SRR6231080,TPM_SRR6231081,TPM_SRR6231082,TPM_SRR6231083,TPM_SRR6231084,TPM_SRR6231085,TPM_SRR6231086,TPM_SRR6231087,TPM_SRR6231088,TPM_SRR6231089,counts_SRR6231076,counts_SRR6231077,counts_SRR6231078,counts_SRR6231079,counts_SRR6231080,counts_SRR6231081,counts_SRR6231082,counts_SRR6231083,counts_SRR6231084,counts_SRR6231085,counts_SRR6231086,counts_SRR6231087,counts_SRR6231088,counts_SRR6231089
Gene,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003.14,0.904746,1.728894,2.916374,1.672743,1.783191,0.839223,2.240574,0.740899,2.093251,0.184064,1.386037,0.135561,0.098742,2.971087,31.812,60.686,121.274,34.846,69.902,25.150,66.644,22.725,48.893,10.447,49.374,9.590,7.226,80.811
ENSG00000000005.6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
ENSG00000000419.12,49.583837,42.782856,70.891017,45.358546,61.096438,49.674101,42.417625,49.326380,43.877557,53.120515,50.326623,34.554971,50.109034,69.002702,574.999,565.000,1041.999,516.999,789.000,486.000,510.001,493.000,366.001,598.999,603.000,453.000,656.000,894.000
ENSG00000000457.14,6.224620,5.048875,0.223772,4.225084,3.920400,8.449201,6.421131,2.539701,4.956359,6.579424,6.098504,5.802447,3.945474,7.202070,279.780,326.948,27.987,241.128,222.033,340.791,343.188,118.188,163.229,301.322,318.438,263.851,257.803,320.005
ENSG00000000460.17,0.098994,2.188371,5.427989,4.538087,2.489981,3.327152,0.916714,2.425374,1.152087,5.663703,3.142772,1.237089,4.775197,3.898852,5.915,76.377,95.173,131.395,75.607,74.676,43.962,51.531,33.391,112.769,38.606,55.939,121.780,109.654


In [11]:
agg.loc[:,TPM_cols].to_csv('TPM.csv')
agg.loc[:,counts_cols].to_csv('counts.csv')